Instalace:
    [Paraview download](https://www.paraview.org/download/)


Paraview má dvě části:
    
    Client (GUI, Python) - nemá vlastní data a filtry, ale jen jejich Proxy
    Server - má data, může být distribuovaný na více strojích přes MPI
    
Skriptování na Clientu:
    - lze zadávat vizualizační operace a filtry, ale ne přímo manipulovat s daty
    - lze také zasahovat do GUI, přidávat funkce a pod.

Skriptování na Serveru:
    - PythonCalculator
    - ProgrammableFilter
    
Paraview je postaveno na knihovne VTK, ktera take ma Python rozhrani.    

Zdroje:

- [Python Client Scripting](https://www.paraview.org/Wiki/ParaView/Python_Scripting)
- [Python doc. module simple](https://www.paraview.org/ParaView/Doc/Nightly/www/py-doc/index.html)
- [VTK Python examples](https://www.vtk.org/Wiki/VTK/Examples/Python)    

# 4.2.1 Client side scripting

## Simple interactive script

Script 'pv_sphere.py'
```
from paraview.simple import *
import vtk

# Create a view
view = CreateRenderView()

# Initialize a new interactive Window
iren = vtk.vtkRenderWindowInteractor()
iren.SetInteractorStyle(vtk.vtkInteractorStyleTrackball())
iren.SetRenderWindow(view.GetRenderWindow())
iren.Initialize()

#################
# Build pipeline

# Create a new sphere proxy on the active connection and register it
# in the sources group.
sphere = Sphere(ThetaResolution=8, PhiResolution=32)

# Apply a shrink filter
shrink = Shrink(sphere)

# Turn the visiblity of the shrink object on.
Show(shrink)

# Render the scene
Render()

# Start interaction
iren.Start()
```

# 4.2.2 Server side scripting

## Python Calculator filter

Unlike  the `Calculator` the `Python Calculator` can operate on more inputs. 
This is usefull to compute difference between two fields.

[Python Calculator overview](https://www.paraview.org/Wiki/Python_calculator_and_programmable_filter)

[Python Calculator new overview](https://www.paraview.org/Wiki/index.php?title=Python_Calculator&redirect=no
                                )

## ProgrammableFilter

[ProgrammableFilter examples](https://www.paraview.org/Wiki/Python_Programmable_Filter) - Provides very basic examples, for real work one have to know better the VTK library and figure out how to handle various objects from Python.

[Making a custom filter](https://blog.kitware.com/easy-customization-of-the-paraview-python-programmable-filter-property-panel/) - Tutorial about making a XML file wrapping the Programmablefilter script to make your own user interface of your custom filter.

**Center Data using Numpy**

```
from paraview.vtk import dataset_adapter as DA

# Get access to the input data.
pdi = self.GetInputDataObject(0,0)
# Get the output data object.
pdo = self.GetOutputDataObject(0)
# Copy all fields from input to the output
pdo.CopyAttributes(pdi)

# Get the mesh points from the input.
old_pts = inputs[0].Points
# Compute the shifted points, subtrack the barycenter.
new_pts = old_pts - mean(old_pts,axis=0)
# Create new paraview field.
arr = DA.numpyTovtkDataArray(new_pts, 'newpts')
# Set new coordinates to the output data.
pdo.GetPoints().SetData(arr)
```